In [24]:
from os import listdir
import pandas as pd
import numpy as np

### Remove reads from BAM (Cleaning)

In [9]:
fw = open("runFilterBam.list.txt","w")
folder = '4.q20Files.bacteria/'
for bam in listdir(folder):
    if bam[-3:] == "bam":
        sample = bam.split('.')[0]
        out = bam.replace("bam","clean.bam")
        print >> fw , "picard-tools FilterSamReads I="+folder+bam+" O="+out+" READ_LIST_FILE="+"9.readsForDepletion.bacteria/"+sample+".reads.contaminants.csv FILTER=excludeReadList"
fw.close()

In [10]:
%%bash
chmod +x runFilterBam.list.txt
./runFilterBam.list.txt

OpenJDK 64-Bit Server VM warning: ignoring option MaxPermSize=256m; support was removed in 8.0
[Fri Aug 11 08:00:57 UTC 2017] net.sf.picard.sam.FilterSamReads INPUT=4.q20Files.bacteria/Ab1.10.q20.bam FILTER=excludeReadList [OUTPUT bam will contain reads that are *not* supplied in the READ_LIST_FILE file] READ_LIST_FILE=9.readsForDepletion.bacteria/Ab1.reads.contaminants.csv OUTPUT=Ab1.10.q20.clean.bam    WRITE_READS_FILES=true VERBOSITY=INFO QUIET=false VALIDATION_STRINGENCY=STRICT COMPRESSION_LEVEL=5 MAX_RECORDS_IN_RAM=500000 CREATE_INDEX=false CREATE_MD5_FILE=false
[Fri Aug 11 08:00:57 UTC 2017] Executing as jmoreno@rick.eb.local on Linux 4.4.67 amd64; OpenJDK 64-Bit Server VM 1.8.0_131-8u131-b11-2ubuntu1.16.04.3-b11; Picard version: 1.113() JdkDeflater
INFO	2017-08-11 08:00:57	FilterSamReads	Filtering [presorted=true] Ab1.10.q20.bam -> OUTPUT=Ab1.10.q20.clean.bam [sortorder=coordinate]
INFO	2017-08-11 08:00:58	FilterSamReads	549 SAMRecords written to Ab1.10.q20.clean.bam
[Fri Aug 11

In [ ]:
%%bash
rm *.reads
mkdir 10.afterDepletion
mv *.clean.bam 10.afterDepletion/


### 3.1.3 bam2coverage

In [13]:
fw = open("depth.afterClean.bacteria.sh",'w')
folder = '10.afterDepletion/'
for bam in listdir(folder):
    coverage = bam.replace(".bam",".coverage")
    fw.write("samtools depth "+folder+bam+" > "+coverage+"\n")
fw.close()

In [14]:
%%bash
chmod +x depth.afterClean.bacteria.sh
./depth.afterClean.bacteria.sh
mkdir 10.afterDepletion/coverage
mv *.clean.coverage 10.afterDepletion/coverage/

### Cat coverage files

In [26]:
samples = set()
for fileName in listdir("10.afterDepletion/coverage/"):
    samples.add(fileName.split('.')[0])
fw = open("catCoverageMatrices.afterCleaning.sh","w")
for sample in samples:
    fw.write("cat 10.afterDepletion/coverage/"+sample+"* > "+sample+".all.coverage.csv\n")
fw.close()

In [27]:
%%bash
chmod +x catCoverageMatrices.afterCleaning.sh
./catCoverageMatrices.afterCleaning.sh
mkdir 10.afterDepletion/allCoverage
mv *.coverage.csv 10.afterDepletion/allCoverage/

In [28]:
%%writefile callCoverageMatrix.afterCleaning.sh
ls 10.afterDepletion/allCoverage/*.all.coverage.csv | while read -r file
do
    qsub -v FOO=$file runCoverageMatrix.sh
done

Overwriting callCoverageMatrix.afterCleaning.sh


In [30]:
%%bash
#chmod +x callCoverageMatrix.afterCleaning.sh
#./callCoverageMatrix.afterCleaning.sh
mkdir 10.afterDepletion/coverageMatrices
mv 10.afterDepletion/allCoverage/*.matrix.csv 10.afterDepletion/coverageMatrices

### Sort

In [31]:
#Read lengths file
referenceLengths =  pd.read_csv("0.bacterialGenomes/BacterialGenomes.length.txt",sep='\t',header=None)
referenceLengths.columns = ["REFERENCE","LENGTH"]
referenceLengths.set_index("REFERENCE",inplace=True)
referenceLengths.sort_values(by="LENGTH",inplace=True)
referenceLengths.head()

,LENGTH
REFERENCE,
MECZ01000002.1,21881
CP000452.1,55635
CM003440.1,61247
CP017830.1,91776
CP013211.1,112031


In [33]:
folder = "10.afterDepletion/coverageMatrices/"
for matrix in listdir(folder):
    if "coverage.matrix.csv" in matrix:
        covMatrix = pd.read_csv(folder+matrix,header=0,index_col=0,sep='\t')
        covMatrix = covMatrix.loc[np.logical_not(pd.isnull(covMatrix.index))]
        referenceLengthsSample = referenceLengths.loc[covMatrix.index]
        referenceLengthsSample.sort_values(by="LENGTH",inplace=True)
        covMatrix = covMatrix.loc[referenceLengthsSample.index]
        covMatrix.to_csv(folder+matrix,sep='\t',header=True,index=True)